In [1]:
import speakeasy
import os
import time
import pandas as pd
import numpy as np
from pefile import PEFormatError
from unicorn import UcError

In [2]:
MALWARE_PATH = "../../../data/pe.dataset/PeX86Exe/ransomware/"
files = [x for x in os.listdir(MALWARE_PATH)[0:50]]

In [3]:
reports = {}
timedeltas = []

now = time.time()
for i, file in enumerate(files):
    # calculations
    took = time.time()-now
    timedeltas.append(took)
    now = time.time()

    print(f"{i}/{len(files)}, took: {took:.2f}s", end=" | ")
    se = speakeasy.Speakeasy()
    try:
        module = se.load_module(MALWARE_PATH+file)
        se.run_module(module)
        reports[file] = se.get_report()
    except PEFormatError as ex:
        print("\nfailed ", ex)
    except UcError as ex:
        print("\nfailed ", ex)

print(f"average analysis time per sample: {np.mean(timedeltas)}")

0/50, took: 0.00s | 1/50, took: 0.36s | 2/50, took: 0.33s | 3/50, took: 0.97s | 4/50, took: 1.18s | 5/50, took: 0.52s | 6/50, took: 5.98s | 7/50, took: 0.35s | 8/50, took: 0.36s | 9/50, took: 1.22s | 10/50, took: 1.24s | 11/50, took: 60.34s | 
failed  Invalid memory write (UC_ERR_WRITE_UNMAPPED)
12/50, took: 3.23s | 13/50, took: 0.51s | 14/50, took: 0.83s | 15/50, took: 1.09s | 16/50, took: 0.62s | 17/50, took: 1.00s | 18/50, took: 1.00s | 19/50, took: 1.28s | 20/50, took: 6.01s | 21/50, took: 0.34s | 22/50, took: 0.33s | 23/50, took: 0.94s | 24/50, took: 1.06s | 25/50, took: 1.06s | 26/50, took: 1.06s | 27/50, took: 1.30s | 28/50, took: 0.64s | 29/50, took: 0.63s | 30/50, took: 0.35s | 31/50, took: 0.90s | 32/50, took: 0.93s | 33/50, took: 0.34s | 34/50, took: 0.41s | 35/50, took: 1.35s | 36/50, took: 2.37s | 37/50, took: 1.38s | 38/50, took: 0.90s | 39/50, took: 0.89s | 40/50, took: 0.32s | 41/50, took: 0.90s | 42/50, took: 60.35s | 43/50, took: 1.25s | 44/50, took: 0.34s | 45/50, to

In [4]:
def parse_report(report):
    aa = pd.json_normalize(report)
    ab = pd.json_normalize(aa["entry_points"].iloc[0])
    try:
        if ab["apis"].any():
            #print(ab["apis"])
            ac = pd.json_normalize(ab["apis"].iloc[0])
        else:
            ac = []
    except AttributeError:
        ac = []
    return aa, ab, ac

In [5]:
for i,f in enumerate(files):
    a, b, c = parse_report(reports[f])
    if "error.type" in b.keys():
        print(f," : " ,b["error.type"].iloc[0])
    else:
        print(f, " : success")

674463fcca9e1b97e7cc935e0cedc625ca6ff882ffd1dc616495faf990d4541c  :  unsupported_api
4b18b54359fa9fa61e02d03b494a7b9219388706e74094c1875e47d45ad94f24  :  unsupported_api
24c3804ea876f04eb4e456d9441ddbc1b01221e2258785fc506531751979a932  :  Failed to get character width from function: kernel32.GetVolumeInformation
e347a3172a900049c0821cab63f2c1b2fa32e0dfe55756565cacf0e5c573814b  :  Failed to get character width from function: kernel32.GetVolumeInformation
0029608fc206f08ed0ec885ab4f4e1ee464d72fe2e64c88d4acc74995942c473  :  Failed to get character width from function: kernel32.GetVolumeInformation
b1e1e7cbf94860163175afcfd6f552c6889a84d1e85feb57fc9c3995a1e5a240  :  Failed to get character width from function: kernel32.GetVolumeInformation
83c1c86d504b6826d020b257ab6b1c92b7c933545b2a343b04ad0f4c00c4f50d  :  unsupported_api
a36d4def03d168536434e86e96e472fbde458e5c3653b422005271f0e62a8621  :  unsupported_api
4a56c52a5d1688daf221aa1ee6e344a36d86d5f6dae972db41c890352b75e9d2  :  Failed to get c

KeyError: 'bc9358a81b5cec7210210c7861aae2dda1e9652640a1a0bcd438f433ece4fcc0'

## examine succeeded reports

In [91]:
suceeded_report = reports["f118b1f459eff5bfbf58eb2e6677c593ea47974c6f738e9c4b0fd4287940df40"]
suceeded_aa = pd.json_normalize(suceeded_report["entry_points"])
print(suceeded_aa.columns)
suceeded_aa

Index(['ep_type', 'start_addr', 'ep_args', 'apihash', 'apis', 'ret_val',
       'dynamic_code_segments', 'error.type', 'error.count', 'error.pc',
       'error.last_api'],
      dtype='object')


,ep_type,start_addr,ep_args,apihash,apis,ret_val,dynamic_code_segments,error.type,error.count,error.pc,error.last_api
0,module_entry,0x404663,"[0x4000, 0x4010, 0x4020, 0x4030]",c16d89e1464b49e5533e624f6b5137c2154a9c5dc95333...,"[{'pc': '0x409d4b', 'api_name': 'KERNEL32.GetS...",0x0,[],max_api_count,10000,0x401875,KERNEL32.GetThreadPriority


## examine failed reports

In [92]:
failed_report = reports[list(reports.keys())[0]]
failed_aa = pd.json_normalize(failed_report["entry_points"])
print(failed_aa.columns)
failed_aa

Index(['ep_type', 'start_addr', 'ep_args', 'apihash', 'apis', 'ret_val',
       'dynamic_code_segments', 'error.type', 'error.pc', 'error.address',
       'error.instr', 'error.regs.esp', 'error.regs.ebp', 'error.regs.eip',
       'error.regs.esi', 'error.regs.edi', 'error.regs.eax', 'error.regs.ebx',
       'error.regs.ecx', 'error.regs.edx', 'error.stack', 'error.traceback'],
      dtype='object')


,ep_type,start_addr,ep_args,apihash,apis,ret_val,dynamic_code_segments,error.type,error.pc,error.address,...,error.regs.ebp,error.regs.eip,error.regs.esi,error.regs.edi,error.regs.eax,error.regs.ebx,error.regs.ecx,error.regs.edx,error.stack,error.traceback
0,module_entry,0x650a99,"[0x4000, 0x4010, 0x4020, 0x4030]",e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,[],0x77000f93,[],Failed to get character width from function: k...,0x77000f93,0x77000f93,...,0x01212000,0x77000f93,0x00401d1f,0x00024c15,0x77000f93,0x01211fd8,0x00000003,0x01211fdc,[sp+0x00: 0x00401c77 -> emu.module.24c3804ea87...,"Traceback (most recent call last):\n File ""/u..."


In [95]:
error_cols = set(failed_aa) - set(suceeded_aa)   
error_cols

{'error.address',
 'error.instr',
 'error.regs.eax',
 'error.regs.ebp',
 'error.regs.ebx',
 'error.regs.ecx',
 'error.regs.edi',
 'error.regs.edx',
 'error.regs.eip',
 'error.regs.esi',
 'error.regs.esp',
 'error.stack',
 'error.traceback'}

In [99]:
failed_aa[error_cols]

,error.regs.esp,error.regs.edx,error.address,error.regs.edi,error.regs.ebx,error.instr,error.regs.eax,error.traceback,error.regs.ebp,error.stack,error.regs.ecx,error.regs.esi,error.regs.eip
0,0x01211fb4,0x01211fdc,0x77000f93,0x00024c15,0x01211fd8,"mov edi, edi",0x77000f93,"Traceback (most recent call last):\n File ""/u...",0x01212000,[sp+0x00: 0x00401c77 -> emu.module.24c3804ea87...,0x00000003,0x00401d1f,0x77000f93


In [103]:
print(failed_aa["error.traceback"].iloc[0])

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/speakeasy_emulator-1.5.9-py3.6.egg/speakeasy/windows/winemu.py", line 1168, in handle_import_func
    rv = self.api.call_api_func(mod, func, argv, ctx=default_ctx)
  File "/usr/local/lib/python3.6/site-packages/speakeasy_emulator-1.5.9-py3.6.egg/speakeasy/winenv/api/winapi.py", line 77, in call_api_func
    return func(mod, self.emu, argv, ctx)
  File "/usr/local/lib/python3.6/site-packages/speakeasy_emulator-1.5.9-py3.6.egg/speakeasy/winenv/api/usermode/kernel32.py", line 3587, in GetVolumeInformation
    cw = self.get_char_width(ctx)
  File "/usr/local/lib/python3.6/site-packages/speakeasy_emulator-1.5.9-py3.6.egg/speakeasy/winenv/api/api.py", line 343, in get_char_width
    raise ApiEmuError('Failed to get character width from function: %s' % (name))
speakeasy.errors.ApiEmuError: Failed to get character width from function: kernel32.GetVolumeInformation



In [96]:
success_cols =  set(suceeded_aa) - set(failed_aa)
success_cols

{'error.count', 'error.last_api'}

In [104]:
suceeded_aa[success_cols]

,error.last_api,error.count
0,KERNEL32.GetThreadPriority,10000
